In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

#To display all outputs in a cell rather than the last output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
DF = pd.merge(study_results, mouse_metadata, how='left', on='Mouse ID')

In [2]:
# Checking the number of mice in the DataFrame.
DF['Mouse ID'].value_counts().count()

249

In [3]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
# Note:- As per the description, for a timepoint, there should be only one entry for any mouse


#Duplicate Mice
duplicate_mice_ids = list(DF[DF.duplicated(subset=['Mouse ID','Timepoint'], keep = False)]['Mouse ID'].unique())
duplicate_mice_ids

['g989']

In [4]:
# Optional: Get all the data for the duplicate mouse ID. 

#Extra:- Duplicated Entries are displayed as follows:
#DF[DF.duplicated(subset=['Mouse ID','Timepoint'], keep = False)]

DF[DF['Mouse ID'].isin(duplicate_mice_ids)]
#13 rows for this mouse

,Mouse ID,Timepoint,Tumor Volume (mm3),Metastatic Sites,Drug Regimen,Sex,Age_months,Weight (g)
107,g989,0,45.000000,0,Propriva,Female,21,26
137,g989,0,45.000000,0,Propriva,Female,21,26
329,g989,5,48.786801,0,Propriva,Female,21,26
360,g989,5,47.570392,0,Propriva,Female,21,26
620,g989,10,51.745156,0,Propriva,Female,21,26
681,g989,10,49.880528,0,Propriva,Female,21,26
815,g989,15,51.325852,1,Propriva,Female,21,26
869,g989,15,53.442020,0,Propriva,Female,21,26
950,g989,20,55.326122,1,Propriva,Female,21,26
1111,g989,20,54.657650,1,Propriva,Female,21,26


In [5]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
# Note:- All data corresponding to this mouse is dropped

DF.drop(DF[DF['Mouse ID'].isin(duplicate_mice_ids)].index, axis=0, inplace=True)
#Left with 1880 rows

In [6]:
# Checking the number of mice in the clean DataFrame.
DF['Mouse ID'].value_counts().count()

248

## Summary Statistics

In [34]:
######################################PLEASE READ THIS######################################
#1. Though we are taking a complete data set, it has to be considered as sample NOT population. Reason being, 248 mice represents entire population of mice here to derive the study results
#2. Because of 1, variance formula should be that of sample variance (ddof = 1, losing 1 degree  of freedom as sample mean is derived from the same data)
#3. Because of 1, std deviation formula should be that of sample std deviation (ddof = 1, losing 1 degree  of freedom as sample mean is derived from the same data)
#4. Most importantly, because of 1, SEM is not a relevant statistics here. SEM is the std. error of mean. It is tied to CLT (Central Limit Theorem), which says, the sample mean of "truly" randomly \
####picked ensembles converge to the population mean with std error of mean (SEM) = population std /sqrt(of ensemble size). This is used to derive population std deviation from samples.
####Means, if we know SEM, then std.deviation = SEM*ensemble size. 
####The scipy implementation of SEM is std_dev/sqrt(size of array) is not useful for us. I am using SEM(ddof =1) here, just to match up with std calculation. But, rightfully, we shouldn't take SEM here.

In [53]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straightforward, creating multiple series and putting them all together at the end.

#Create a group based on 'Drug Regimen'
drug_regimen_grp = DF.groupby('Drug Regimen')
mean = drug_regimen_grp['Tumor Volume (mm3)'].mean()
median = drug_regimen_grp['Tumor Volume (mm3)'].median()
variance = drug_regimen_grp['Tumor Volume (mm3)'].var(ddof=1)
std_dev = drug_regimen_grp['Tumor Volume (mm3)'].std(ddof=1)
sem = drug_regimen_grp['Tumor Volume (mm3)'].apply(lambda x: st.sem(x, ddof=1))

#Combine everything to a DF
summary_stats_method1 = pd.concat([mean, median, variance, std_dev, sem], axis=1)
summary_stats_method1.columns = ['mean', 'median', 'variance', 'std_dev', 'sem']
summary_stats_method1

,mean,median,variance,std_dev,sem
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [55]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method produces everything in a single groupby function.


#Create a group based on 'Drug Regimen'
drug_regimen_grp = DF.groupby('Drug Regimen')
#Aggregate based on the summary stats
summary_stats_method2 = drug_regimen_grp.aggregate(mean=('Tumor Volume (mm3)', 'mean'), median=('Tumor Volume (mm3)', 'median'), variance=('Tumor Volume (mm3)', lambda x:np.var(x, ddof=1)), std_dev = ('Tumor Volume (mm3)', lambda x:np.std(x, ddof=1)), sem = (('Tumor Volume (mm3)', lambda x: st.sem(x, ddof=1))))                  
summary_stats_method2



,mean,median,variance,std_dev,sem
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


with 3 points, ybar mean and std are -0.00005, 0.576; SEM is 0.577
The mean of the estimated standard deviations is 0.878
The median is 0.832
The square root of the mean estimated variance is 0.991


In [111]:
y.shape

(10000, 3)

In [115]:
ystd

array([1.30880833, 0.84578685, 0.61590036, ..., 0.9725977 , 0.76862378,
       0.96531931])

In [117]:
ybar.std(ddof=0)

0.5756137817221784

In [17]:
st.sem(DF[DF['Drug Regimen']=='Ceftamin']['Tumor Volume (mm3)'], ddof=1)
import numpy as np
np.std(DF[DF['Drug Regimen']=='Ceftamin']['Tumor Volume (mm3)'], ddof=0)

0.46982053275261093

6.250555650522812

In [14]:
len(DF[DF['Drug Regimen']=='Ceftamin']['Tumor Volume (mm3)'])

178

In [19]:
from scipy.stats import sem
import numpy as np
l = [0,2,4,5,6,7]
print(sem(l))
print(np.std(l, ddof=1)/np.sqrt(len(l)))

1.0645812948447542
1.0645812948447542
